In [1]:
import scsims 
import pandas as pd 
import numpy as np
import os
import anndata as an
from scsims import SIMS 
import scanpy as sc
import anndata as an
from torch.utils.data import DataLoader
from scsims.testing import TestAnndatasetMatrix
import torch
from tqdm import tqdm
from scsims import clean_sample 
import numpy as np
import pandas as pd 
from scsims import SIMSClassifier
from scsims.data import AnnDatasetMatrix
import plotly.express as px 
from scsims import DataModule
import os 
from pytorch_lightning.loggers import WandbLogger
import pytorch_lightning as pl 

Let's try retraining on the raw counts so we can compare raw <--> raw. We'll use my converter to do this

In [26]:
# from bigcsv import to_h5ad

# to_h5ad(
#     file='../data/bhaduri/primary_RAWCOUNTS.txt',
#     outfile='../data/bhaduri/primary_RAWCOUNTS.h5ad',
#     sep='\t',
#     sparsify=True,
#     chunksize=100,
#     quiet=False,
#     index_col='Unnamed: 0',
# )

In [4]:
allen = pd.read_csv('../data/benchmark/human_labels_clean.csv')
bhaduri = pd.read_csv('../data/bhaduri/primary_labels_clean.csv')

In [5]:
allen_X = an.read_h5ad('../data/benchmark/human_lognorm.h5ad')
bhaduri_X = an.read_h5ad('../data/bhaduri/primary_RAWCOUNTS_LOGNORM_T.h5ad')

In [6]:
traingenes = [x.upper() for x in bhaduri_X.var['index'].values]
testgenes = [x.upper() for x in allen_X.var.index]

In [7]:
refgenes = list(set(traingenes).intersection(testgenes))

In [8]:
mapping = {
    'VIP': ['SST-MGE1', 'MGE2'],
    'LAMP5': ['SST-MGE1', 'MGE2'],
    'IT': ['Upper Layer Occipital', 'Upper Layer', 'Upper Layer PFC', 'Parietal and Temporal', 'Layer IV'],
    'PAX6': ['oRG', 'vRG', 'tRG', 'oRG/Astrocyte'],
    'Oligodendrocyte': ['oRG', 'vRG', 'tRG', 'oRG/Astrocyte', 'OPC'],
    'Astrocyte': ['oRG', 'vRG', 'tRG', 'oRG/Astrocyte'],
    'L5/6 IT Car3': ['Upper Layer Occipital', 'Upper Layer', 'Upper Layer PFC', 'Parietal and Temporal', 'Layer IV'],
    'L5/6 NP': ['Upper Layer Occipital', 'Upper Layer', 'Upper Layer PFC', 'Parietal and Temporal'], # unkown
    'SST': ['SST-MGE1'],
    'L6 CT': ['Layer VI Occipital', 'Deep Layer', 'Layer VI Pan-area', 'Parietal and Temporal'],
    'OPC': ['OPC'],
    'PVALB': ['SST-MGE1', 'MGE2'], # check this one 
    'L6b': ['Layer VI Occipital', 'Deep Layer', 'Layer VI Pan-area', 'Parietal and Temporal'],
    'Microglia': ['Microglia'],
    'L5 ET': ['Layer VI Occipital', 'Deep Layer', 'Layer VI Pan-area', 'Parietal and Temporal'],
    'Pericyte': ['Mural'], # check this one 
    'Endothelial': ['Endothelial'],
    'L4 IT': ['Layer IV'],
    'VLMC': ['oRG', 'Layer VI Occipital', 'Newborn', 'vRG', 'early', 'Mural',
       'IPC/newborn', 'OPC', 'Upper Layer Occipital', 'IPC-new',
       'Cajal Retzius', 'Upper Layer', 'Layer VI Pan-area', 'Outlier',
       'SST-MGE1', 'IPC-div1', 'PFC', 'Upper Layer PFC', 'Microglia',
       'Deep Layer', 'late', 'Parietal and Temporal', 'IPC-div2',
       'Layer IV', 'MGE2', 'Endothelial', 'oRG/Astrocyte', 'tRG'],
}

In [9]:
class UploadCallback(pl.callbacks.Callback):
    def __init__(
        self, 
        path: str, 
        desc: str, 
        upload_path='model_checkpoints',
        epochs: int=1,
    ) -> None:
        super().__init__()
        self.path = path 
        self.desc = desc
        self.upload_path = upload_path
        self.epochs = epochs 

    def on_train_epoch_end(self, trainer, pl_module):
        epoch = trainer.current_epoch

        if epoch % self.epochs == 0 and epoch > 0: # Save every ten epochs
            checkpoint = f'checkpoint-{epoch}-desc-{self.desc}.ckpt'
            trainer.save_checkpoint(os.path.join(self.path, checkpoint))
            print(f'Saving checkpoint at epoch {epoch}')
            
module = DataModule(
    datafiles=['../data/bhaduri/primary_RAWCOUNTS_T.h5ad'],
    labelfiles=['../data/bhaduri/primary_labels_clean.csv'],
    class_label='categorical_Subtype',
    index_col='cell',
    batch_size=16,
    num_workers=0,
    deterministic=True,
    normalize=True,
    currgenes=traingenes,
    refgenes=refgenes,
)

module.prepare_data()
module.setup()

wandb_logger = WandbLogger(
    project=f"Bhaduri human organoid model",
)

lr_callback = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

upload_callback = UploadCallback(
    path='checkpoints',
    desc=f'local_bhaduri_allen_raw'
)

early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=6,
)

trainer = pl.Trainer(
    gpus=(1 if torch.cuda.is_available() else 0),
    auto_lr_find=False,
    logger=wandb_logger,
    max_epochs=500,
    gradient_clip_val=0.5,
    callbacks=[
        lr_callback, 
        upload_callback,
        early_stopping_callback,
    ],
    stochastic_weight_avg=True,
)


model = SIMSClassifier(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    weights=module.weights,
)

print(f'Input dim and output dim are {module.num_features} / {module.num_labels}')

trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


wandb: Currently logged in as: jlehrer1. Use `wandb login --relogin` to force relogin


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:64: LightningDeprecationWarning: Setting `Trainer(stochastic_weight_avg=True)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.stochastic_weight_avg.StochasticWeightAveraging` directly to the Trainer's `callbacks` argument instead.
  rank_zero_deprecation(
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Initializing network
Initializing explain matrix
Input dim and output dim are 18427 / 28
Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights



  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 1.2 M 
-----------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.888     Total estimated model params size (MB)


...

Validation: 0it [00:00, ?it/s]

Saving checkpoint at epoch 24


Validation: 0it [00:00, ?it/s]

Saving checkpoint at epoch 25


wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.


In [77]:
from scsims.data import CollateLoader 
from scsims.testing import TestAnndatasetMatrix

testdataset = TestAnndatasetMatrix(
    allen_X[allen['cell'].values, :].X,
)

testloader = CollateLoader(
    dataset=testdataset, 
    batch_size=4, 
    num_workers=0, 
    refgenes=refgenes, 
    currgenes=testgenes,
    normalize=True,
)

In [17]:
from functools import partial
tqdm = partial(tqdm, position=0, leave=True)

In [18]:
model = SIMSClassifier.load_from_checkpoint(
#     'checkpoints/checkpoint-8-desc-local_bhaduri_allen_human_human.ckpt',
    'checkpoints/',
    input_dim=module.num_features,
    output_dim=module.num_labels,
)

Initializing network
Initializing explain matrix


In [78]:
preds = []

model.eval()
with torch.no_grad():
    for X in tqdm(testloader):
        res, _ = model(X)
        
        _, top_preds = res.topk(3, axis=1) # to get indices
        preds.extend(top_preds.numpy())
        
preds = pd.DataFrame(preds)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 11858/11858 [07:51<00:00, 25.13it/s]


In [21]:
preds.shape

(47432, 3)

In [22]:
len(allen)

47432

In [24]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit([x for x in bhaduri['categorical_categorical_Subtype'].unique()])

In [27]:
allen['categorical_subclass_label']

0                    VIP
1                  LAMP5
2                  LAMP5
3                    VIP
4                    VIP
              ...       
47427          Astrocyte
47428                 IT
47429              LAMP5
47430    Oligodendrocyte
47431    Oligodendrocyte
Name: categorical_subclass_label, Length: 47432, dtype: object

In [79]:
allen['predicted'] = le.inverse_transform(preds.loc[:, 0].values)

In [74]:
import scanpy

scanpy.pp.log1p(allen_X)

In [76]:
allen_X.write_h5ad('../data/benchmark/human_lognorm.h5ad')

In [80]:
acc = 0
for truth, pred in zip(allen["categorical_subclass_label"], allen["predicted"]):
    options = mapping[truth]
    if pred in options:
        acc += 1

acc/len(allen)

0.08209647495361781

In [81]:
allen_X.X.todense()

matrix([[0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.6931472],
        [0.       , 0.       , 0.       , ..., 6.618739 , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [0.       , 0.       , 0.       , ..., 4.8675346, 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 5.9322453, 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]], dtype=float32)

In [83]:
bhaduri_X.X.todense()

matrix([[2.8466673, 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 1.8506957, 0.       , ..., 0.       , 0.       ,
         0.       ],
        ...,
        [2.0499473, 2.2653387, 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ],
        [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
         0.       ]], dtype=float32)

In [43]:
for label in allen['categorical_subclass_label'].unique():
    temp = allen[allen['categorical_subclass_label'] == label]
    acc = 0
    for pred in temp["predicted"]:
        options = mapping[label]
        if pred in options:
            acc += 1

    print(f'Accuracy for label {label} is {acc/len(temp)}')

Accuracy for label VIP is 0.9971695442966317
Accuracy for label LAMP5 is 0.9987674609695973
Accuracy for label IT is 0.6470399706973123
Accuracy for label PAX6 is 0.0
Accuracy for label Oligodendrocyte is 0.02383419689119171
Accuracy for label Astrocyte is 0.1988205560235889
Accuracy for label L5/6 IT Car3 is 0.42924976258309594
Accuracy for label L5/6 NP is 0.8296568627450981
Accuracy for label SST is 0.062340966921119595
Accuracy for label L6 CT is 0.6557120500782473
Accuracy for label OPC is 0.96248382923674
Accuracy for label PVALB is 0.9960714285714286
Accuracy for label L6b is 0.6601851851851852
Accuracy for label Microglia is 0.8106666666666666
Accuracy for label L5 ET is 0.8860759493670886
Accuracy for label Pericyte is 0.1875
Accuracy for label Endothelial is 0.2
Accuracy for label L4 IT is 0.006174496644295302
Accuracy for label VLMC is 1.0


In [53]:
px.bar(allen['predicted'].value_counts(), title='Distribution of predicted values')

wandb: Network error (ConnectionError), entering retry loop.


In [48]:
bhaduri['categorical_categorical_Subtype'].value_counts()

Newborn                  33178
SST-MGE1                 19596
Upper Layer Occipital    13099
vRG                      11285
IPC/newborn               9869
Layer VI Occipital        9442
Upper Layer               8936
oRG                       8745
Parietal and Temporal     7981
Layer VI Pan-area         7842
Upper Layer PFC           6988
early                     4112
Layer IV                  3725
Deep Layer                3714
PFC                       3361
oRG/Astrocyte             3010
Outlier                   2933
OPC                       1888
late                      1887
IPC-div2                  1496
IPC-div1                  1398
Microglia                 1291
MGE2                      1013
IPC-new                    969
Mural                      363
Endothelial                271
Cajal Retzius              268
tRG                         37
Name: categorical_categorical_Subtype, dtype: int64

In [63]:
bhaduri_no_pandt_mge2 = bhaduri[~bhaduri['categorical_categorical_Subtype'].isin(['Parietal and Temporal', 'MGE2'])]
bhaduri_no_pandt_mge2['Subtype'] = LabelEncoder().fit_transform(bhaduri_no_pandt_mge2['categorical_categorical_Subtype'])

/var/folders/pd/jsjcl0fn7w57s5mfr34b20pm0000gn/T/ipykernel_6196/2387576657.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [67]:
bhaduri_no_pandt_mge2.to_csv('../data/bhaduri/bhaduri_no_pandt_mge2_labels.csv', index=False)

In [68]:
module = DataModule(
    datafiles=['../data/bhaduri/primary_T.h5ad'],
    labelfiles=['../data/bhaduri/bhaduri_no_pandt_mge2_labels.csv'],
    class_label='Subtype',
    index_col='cell',
    batch_size=16,
    num_workers=0,
    deterministic=True,
    normalize=True,
    currgenes=traingenes,
    refgenes=refgenes,
)

module.prepare_data()
module.setup()

wandb_logger = WandbLogger(
    project=f"Bhaduri human organoid model",
)

lr_callback = pl.callbacks.LearningRateMonitor(logging_interval='epoch')

upload_callback = UploadCallback(
    path='checkpoints',
    desc=f'local_bhaduri_allen_human_no_pandt_mge2'
)

early_stopping_callback = pl.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=6,
)

trainer = pl.Trainer(
    gpus=(1 if torch.cuda.is_available() else 0),
    auto_lr_find=False,
    logger=wandb_logger,
    max_epochs=500,
    gradient_clip_val=0.5,
    callbacks=[
        lr_callback, 
        upload_callback,
        early_stopping_callback,
    ],
    stochastic_weight_avg=True,
)


model = SIMSClassifier(
    input_dim=module.num_features,
    output_dim=module.num_labels,
    weights=module.weights,
)

print(f'Input dim and output dim are {module.num_features} / {module.num_labels}')

trainer.fit(model, datamodule=module)

Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:347: UserWarning:

There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/callback_connector.py:64: LightningDeprecationWarning:

Setting `Trainer(stochastic_weight_avg=True)` is deprecated in v1.5 and will be removed in v1.7. Please pass `pytorch_lightning.callbacks.stochastic_weight_avg.StochasticWeightAveraging` directly to the Trainer's `callbacks` argument instead.

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Initializing network
Initializing explain matrix
Input dim and output dim are 16607 / 26
Creating train/val/test DataLoaders...
Done, continuing to training.
Calculating weights


/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:611: UserWarning:

Checkpoint directory /Users/julian/Documents/Projects/scanvi_benchmark/notebooks/Bhaduri human organoid model/zo6mmcei/checkpoints exists and is not empty.


  | Name    | Type   | Params
-----------------------------------
0 | network | TabNet | 1.1 M 
-----------------------------------
1.1 M     Trainable params
0         Non-trainable params
1.1 M     Total params
4.408     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:240: PossibleUserWarning:

The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/torchmetrics/utilities/prints.py:36: UserWarning:

No positive samples in targets, true positive value should be meaningless. Returning zero tensor in true positive score

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/scsims/model.py:207: RuntimeWarning:

invalid value encountered in true_divide

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/scsims/model.py:208: RuntimeWarning:

invalid value encountered in true_divide

/Users/julian/miniconda3/envs/sims/lib/python3.9/site-packages/scsims/model.py:209

Training: 0it [00:00, ?it/s]

ValueError: Expected more than 1 value per channel when training, got input size torch.Size([1, 16607])

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error (ConnectionError), entering retry loop.
